In [2]:
import numpy as np
from scipy.optimize import fsolve, brentq
from scipy.linalg import lu
import matplotlib.pyplot as plt
import pandas as pd 
import seaborn as sns 
from math import sin,cos,pi 
from numpy.linalg import lstsq

## 🧩 Task 1: Root-Finding — Solving for Incident Angle \( \theta \)

In this task, we estimate the angle of incidence \( \theta \) from radar phase data using the **Newton-Raphson method**.

### ⚙️ Method:
- **Newton-Raphson algorithm**
- Initial guess: within \([-1.0, 0.0]\)
- Tolerance: \(10^{-6}\)
- True root: \(-0.523598826\)

### ✅ Output:
- Rapid convergence within 5 iterations.
- Final root matches the true value with high accuracy.
- Error tracking table shows decreasing residuals.


In [26]:
def radar_func(theta,C): 
    """
    Menghitung nilai fungsi untuk akar interpretasi dari radar. 
    
    Persamaan f(theta) = C * sin(theta) + pi = 0,
    dimana C = lambda * R / (4 * pi * ht * hr).
    """
    return C * np.sin(theta) + np.pi 
def df_radar_func(theta): 
    return C * cos(theta) 

In [33]:
C = 2 * np.pi 
a_interval = -1.0 
b_interval = 0.0 

tolerance = 1e-6 
max_iterations = 100 
true_root = -0.523598826

In [34]:
def table_open(iterates): 
    rows = [] 
    for i, x in enumerate(iterates): 
        if i == 0: 
            approx_error = None 
        else: 
            approx_error = abs(iterates[i]-iterates[i-1]) / abs(iterates[i]) * 100 
        true_error = abs(x - true_root) / abs(true_root) * 100 
        rows.append({
            "Iteration" : i, 
            "x" : x, 
            "error_a (%)" : approx_error,
            "error_t (&)" : true_error
        })
    return rows 

In [35]:
def newton_raphson(x0): 
    iterates = [x0]
    for i in range(max_iterations): 
        f_val = radar_func(iterates[-1],C)
        df_val = df_radar_func(iterates[-1]) 
        if df_val == 0: 
            raise ZeroDivisionError("Turunan sama dengan 0") 
        x_new = iterates[-1] - f_val / df_val 
        iterates.append(x_new)
        if abs(radar_func(x_new,C)) < tolerance or abs(x_new - iterates[-2]) < tolerance: 
            break
    return iterates 

In [36]:
x0_newton = 0.73 
iters_newton = newton_raphson(x0_newton) 
table_newton = table_open(iters_newton) 
df_newton = pd.DataFrame(table_newton) 

In [37]:
display(df_newton) 

,Iteration,x,error_a (%),error_t (&)
0,0,0.730000,NaN,239.419717
1,1,-0.835901,187.330875,59.645391
2,2,-0.475129,75.931458,9.257052
3,3,-0.522976,9.148920,0.119021
4,4,-0.523599,0.118990,0.000031


## 🧮 Task 2: Solving Linear System — Extracting Path Delays

Here, we solve for signal path delays τ  using a system of equations derived from radar multipath signal components.

    Solves the linear system Ax = b using the Gauss-Seidel iterative method,
    tracking iteration history and approximate error.

    Args:
        A (numpy.ndarray): The coefficient matrix.
        b (numpy.ndarray): The constant vector.
        x_init (numpy.ndarray): The initial guess for the solution vector.
        max_iterations (int): The maximum number of iterations allowed.
        tolerance (float): The convergence tolerance based on max absolute change.

    Returns:
        tuple: A tuple containing:
            - solution (numpy.ndarray): The computed solution vector.
            - iterations_completed (int): The number of iterations performed.
            - combined_df (pandas.DataFrame): DataFrame tracking x values and
                                              approximate error per iteration.

    Raises:
        ValueError: If matrix A is not square or dimensions mismatch.
        ZeroDivisionError: If a zero diagonal element is encountered.

In [39]:
#Function for checking, is matrix diagonallay dominant 

def check_diaDom(A): 
    n = len(A) 
    for i in range (n): 
        diag_val = abs(A[i][i]) #Get absoulute value for diagonal element 
        row_sum = sum(abs(A[i][j]) for j in range(n) if j != i) 
    if diag_val < row_sum:
        #Not diagnallay dominant 
        return False 
    return True 

In [ ]:
def GS_method(A, b, x_init, max_iterations=100, tolerance=1e-10):
    n = len(A) 

    if A.shape[0] != n or A.shape[1] != n: 
        raise ValueError("Matrix must be in square") 
    if b.shape[0] != n:
        raise ValueError("Vector b must have the same number of rows as matrix A") 
    if x_init.shape[0] != n: 
        raise ValueError